<a href="https://colab.research.google.com/github/ozanryo/air-kerma-hvl-prediction-AI/blob/main/Gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mounting Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Melakukan Install Pydicom**
---

In [ ]:
pip install pydicom

     |████████████████████████████████| 35.5MB 1.3MB/s 


# **Import Library**
---

In [ ]:
import pydicom as dicom
import os

#Preprocessing Dataset dan Membangun Neural Network
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from skimage.feature import greycomatrix, greycoprops

#Pengukuran Performa Machine Learning
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn import metrics
from keras.utils import plot_model
from sklearn.utils import shuffle

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score

# **Membuat Fungsi**
---

**Membuat Fungsi Input dan Output**

In [ ]:
def Input(Data):
    X = Data.iloc[:, 0:4].values
    labelencoder_X_1 = LabelEncoder()
    
    X[:, 3] = labelencoder_X_1.fit_transform(X[:, 3])
    ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(categories='auto'), [3])], 
                               remainder='passthrough')
    X = ct.fit_transform(X)
    
    #Standard Scaler()
    sc = StandardScaler()
    X[:, 3:6] = sc.fit_transform(X[:, 3:6])
    X[:, 3:6] = sc.transform(X[:, 3:6])
    return X

In [ ]:
def Labels(Data):
    y = Data.iloc[:, 4:7].values
    
    labelencoder_y_1 = LabelEncoder()
    y[:,0] = labelencoder_y_1.fit_transform(y[:,0])
    labelencoder_y_3 = LabelEncoder()
    y[:,1] = labelencoder_y_3.fit_transform(y[:,1])
    
    ct1 = ColumnTransformer([('one_hot_encoder', OneHotEncoder(categories='auto'), [0, 1])], 
                               remainder='passthrough')
    
    y = ct1.fit_transform(y) 
    return y

**Membuat Fungsi Pembuat Model ANN**

In [ ]:
def neural_net(n_layers, inputs, units, activation, output_activation, optimizer):
    if isinstance(units, list):
        assert len(units) == n_layers
    else:
        units = [units] * n_layers
        
    classifier = Sequential()
 
    # Adds first hidden layer with input_dim parameter
    classifier.add(Dense(units = units[0],
                         input_dim = inputs,
                         activation = activation,
                         kernel_initializer = 'glorot_uniform',
                         name = 'h1'))
    
    # Adds remaining hidden layers
    for i in range(2, n_layers + 1):
        classifier.add(Dense(units = units[i-1], 
                        activation = activation, 
                        kernel_initializer = 'glorot_uniform', 
                        name = 'h{}'.format(i)))
    
    # Adds output layer
    classifier.add(Dense(units = y_test_folds.shape[1], 
                         activation = output_activation, 
                         kernel_initializer='glorot_uniform', name='o'))
 
    # Compiles the model
    classifier.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', 'mean_squared_error'])
 
    return classifier

**Membuat Model Umum untuk dilakukan GridSearch**

In [ ]:
def create_model(n_layers=1, units=10, activation='sigmoid', output_activation='sigmoid', optimizer='Adam'):
    if isinstance(units, list):
        assert len(units) == n_layers
    else:
        units = [units] * n_layers
        
    classifier = Sequential()
 
    # Adds first hidden layer with input_dim parameter
    classifier.add(Dense(units = units[0],
                         input_dim = 3,
                         activation = activation,
                         kernel_initializer = 'glorot_uniform',
                         name = 'h1'))
    
    # Adds remaining hidden layers
    for i in range(2, n_layers + 1):
        classifier.add(Dense(units = units[i-1], 
                        activation = activation, 
                        kernel_initializer = 'glorot_uniform', 
                        name = 'h{}'.format(i)))
    
    # Adds output layer
    classifier.add(Dense(units = 6, 
                         activation = output_activation, 
                         kernel_initializer='glorot_uniform', name='o'))
 
    # Compiles the model
    classifier.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', 'mean_squared_error'])
 
    return classifier

**Membuat Fungsi Plotting**

In [ ]:
def plot_result(model):
    # Plot Loss dari Model
    plot_loss = plt.plot(model.history['loss'], label='Train')
    plot_loss = plt.plot(model.history['val_loss'], label='Validation')
    plot_loss = plt.title('Mean Absolute Error')
    plot_loss = plt.grid(True)
    plot_loss = plt.ylabel('Loss Value')
    plot_loss = plt.xlabel('No. epoch')
    plot_loss = plt.legend(loc="upper right")
    plot_loss = plt.show()

    # Plot Accuracy dari Model
    plot_acc = plt.plot(model.history['accuracy'], label='Train')
    plot_acc = plt.plot(model.history['val_accuracy'], label='Validation')
    plot_acc = plt.title('Accuracy')
    plot_acc = plt.grid(True)
    plot_acc = plt.ylabel('Accuracy Value')
    plot_acc = plt.xlabel('No. epoch')
    plot_acc = plt.legend(loc="upper right")
    plot_acc = plt.show()

    # Plot Mean Squared Error dari Model
    plot_MSE = plt.plot(model.history['mean_squared_error'], label='MSE Train')
    plot_MSE = plt.plot(model.history['val_mean_squared_error'], label='MSE Validation')
    plot_MSE = plt.title('Mean Squared Error')
    plot_MSE = plt.grid(True)
    plot_MSE = plt.ylabel('MSE value')
    plot_MSE = plt.xlabel('No. epoch')
    plot_MSE = plt.legend(loc="upper left")
    plot_MSE = plt.show()
    
    return plot_loss, plot_acc, plot_MSE

**Membuat Fungsi Pengukur Performa Machine Learning**

In [ ]:
def Precision_Classification_Kerma(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    ## Prediction on Kerma
    y_pred_Kerma = y_pred[:, 0:3]
    y_pred_Kerma = np.argmax(y_pred_Kerma, axis=1)
     
    ## Take a test on Kerma
    y_test_Kerma = y_test[:, 0:3]
    y_test_Kerma = np.argmax(y_test_Kerma, axis=1)
    
    Precision = precision_score(y_test_Kerma, y_pred_Kerma, average='micro')
    return Precision

def Precision_Classification_HVL(model, X_test, y_test):
    y_pred = model.predict(X_test)
     
    ## Prediction on HVL
    y_pred_HVL = y_pred[:, 3:6]
    y_pred_HVL = np.argmax(y_pred_HVL, axis=1)
     
    ## Take a test on HVL
    y_test_HVL = y_test[:, 3:6]
    y_test_HVL = np.argmax(y_test_HVL, axis=1)
    
    Precision = precision_score(y_test_HVL, y_pred_HVL, average='micro')
    return Precision

def Recall_Classification_Kerma(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    ## Prediction on Kerma
    y_pred_Kerma = y_pred[:, 0:3]
    y_pred_Kerma = np.argmax(y_pred_Kerma, axis=1)
     
    ## Take a test on Kerma
    y_test_Kerma = y_test[:, 0:3]
    y_test_Kerma = np.argmax(y_test_Kerma, axis=1)
    
    Recall = recall_score(y_test_Kerma, y_pred_Kerma, average='micro')
    return Recall

def Recall_Classification_HVL(model, X_test, y_test):
    y_pred = model.predict(X_test)
     
    ## Prediction on HVL
    y_pred_HVL = y_pred[:, 3:6]
    y_pred_HVL = np.argmax(y_pred_HVL, axis=1)
     
    ## Take a test on HVL
    y_test_HVL = y_test[:, 3:6]
    y_test_HVL = np.argmax(y_test_HVL, axis=1)
    
    Recall = recall_score(y_test_HVL, y_pred_HVL, average='micro')
    return Recall

def f1Score_Classification_Kerma(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    ## Prediction on Kerma
    y_pred_Kerma = y_pred[:, 0:3]
    y_pred_Kerma = np.argmax(y_pred_Kerma, axis=1)
     
    ## Take a test on Kerma
    y_test_Kerma = y_test[:, 0:3]
    y_test_Kerma = np.argmax(y_test_Kerma, axis=1)
    
    f1 = f1_score(y_test_Kerma, y_pred_Kerma, average='micro')
    return f1

def f1Score_Classification_HVL(model, X_test, y_test):
    y_pred = model.predict(X_test)
     
    ## Prediction on HVL
    y_pred_HVL = y_pred[:, 3:6]
    y_pred_HVL = np.argmax(y_pred_HVL, axis=1)
     
    ## Take a test on HVL
    y_test_HVL = y_test[:, 3:6]
    y_test_HVL = np.argmax(y_test_HVL, axis=1)
    
    f1 = f1_score(y_test_HVL, y_pred_HVL, average='micro')
    return f1

def Accuracy_Classification_Kerma(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    ## Prediction on Kerma
    y_pred_Kerma = y_pred[:, 0:3]
    y_pred_Kerma = np.argmax(y_pred_Kerma, axis=1)
     
    ## Take a test on Kerma
    y_test_Kerma = y_test[:, 0:3]
    y_test_Kerma = np.argmax(y_test_Kerma, axis=1)
    
    acc = accuracy_score(y_test_Kerma, y_pred_Kerma)
    return acc

def Accuracy_Classification_HVL(model, X_test, y_test):
    y_pred = model.predict(X_test)
     
    ## Prediction on HVL
    y_pred_HVL = y_pred[:, 3:6]
    y_pred_HVL = np.argmax(y_pred_HVL, axis=1)
     
    ## Take a test on HVL
    y_test_HVL = y_test[:, 3:6]
    y_test_HVL = np.argmax(y_test_HVL, axis=1)
    
    acc = accuracy_score(y_test_HVL, y_pred_HVL)
    return acc

**Plot Confusion Matrix**

In [ ]:
def Conf_Mat(X_test_folds, y_test_folds):
  import matplotlib.pyplot as plt
  from sklearn.metrics import ConfusionMatrixDisplay
  from sklearn.utils.multiclass import unique_labels

  y_pred = NN.predict(X_test_folds)
    
  ## Prediction on Kerma
  y_pred_Kerma = y_pred[:, 0:3]
  y_pred_Kerma = np.argmax(y_pred_Kerma, axis=1)
     
  ## Take a test on Kerma
  y_test_Kerma = y_test_folds[:, 0:3]
  y_test_Kerma = np.argmax(y_test_Kerma, axis=1)

  ## Prediction on HVL
  y_pred_HVL = y_pred[:, 3:6]
  y_pred_HVL = np.argmax(y_pred_HVL, axis=1)
     
  ## Take a test on HVL
  y_test_HVL = y_test_folds[:, 3:6]
  y_test_HVL = np.argmax(y_test_HVL, axis=1)

  

  titles_options = [("Confusion Matrix on Air Kerma Predictions", y_pred_Kerma, y_test_Kerma, 'Air Kerma A', 'Air Kerma B', 'Air Kerma C'),
                    ("Confusion Matrix on Half Value Layer Predictions", y_pred_HVL, y_test_HVL, 'HVL A', 'HVL B', 'HVL C')]
      
  for title, pred_y, test_y, A, B, C in titles_options:
      conf_mat = metrics.confusion_matrix(pred_y, test_y)
      disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat,
                                    display_labels=[(A), (B), (C)])
      
      disp = disp.plot(include_values=True,
                       cmap=plt.cm.Blues)
      
      disp.ax_.set_title(title)
      plt.show()

**Melakukan Prediksi Data Image**

In [ ]:
def Prediction_Kerma(Prediction_result):
  if Prediction_result == 0:
    print('The Prediction Kerma is on A')
  elif Prediction_result == 1:
    print('The Prediction Kerma is on B')
  elif Prediction_result == 2:
    print('The Prediction Kerma is on C')

def Prediction_HVL(Prediction_result):
  if Prediction_result == 0:
    print('The Prediction HVL is on A')
  elif Prediction_result == 1:
    print('The Prediction HVL is on B')
  elif Prediction_result == 2:
    print('The Prediction HVL is on C')

In [ ]:
def False_Prediction_Kerma(False_1, False_2, real_kVp, False_kVp_1, False_kVp_2):
  if False_1 == np.argmax(real_kVp):
    print('False Prediction on real Kerma detected')
  elif False_1 == np.argmax(False_kVp_1):
    print('Prediction on another Kerma is detected')
  else :
    print('Kerma Prediction is accurate')
  
  if False_2 == np.argmax(real_kVp):
    print('False Prediction on real Kerma detected')
  elif False_2 == np.argmax(False_kVp_2):
    print('Prediction on another Kerma is detected')
  else :
    print('Kerma Prediction is accurate')

def False_Prediction_HVL(False_1, False_2, real_kVp, False_kVp_1, False_kVp_2):
  if False_1 == np.argmax(real_kVp):
    print('False Prediction on real HVL detected')
  elif False_1 == np.argmax(False_kVp_1):
    print('Prediction on another HVL is detected')
  else :
    print('HVL Prediction is accurate')
  
  if False_2 == np.argmax(real_kVp):
    print('False Prediction on real HVL detected')
  elif False_2 == np.argmax(False_kVp_2):
    print('Prediction on another HVL is detected')
  else :
    print('HVL Prediction is accurate')

**Membuat fungsi untuk Pengambilan Gambar dengan Koordinat utama pada tengah Image**

In [ ]:
def crop_center(img, cropx, cropy):
  y,x = img.shape
  startx = x//2-(cropx//2)
  starty = y//2-(cropy//2)    
  return img[starty:starty+cropy,startx:startx+cropx]

**Menggandakan Array**

In [ ]:
def multiply_array(arr, n):
    result = np.zeros(shape=(n*len(arr), 1))
    idx = 0
    for _ in range(n):
      for element in arr:
        result[idx, 0] = int(element)
        idx += 1
    return result

**Menggandakan Data**

In [ ]:
def multiply_data_kVp(arr, n):
  count = n

  a = np.asarray(arr[:, 0])
  b = np.asarray(arr[:, 1])
  c = np.asarray(arr[:, 2])

  a = multiply_array(a, count)
  b = multiply_array(b, count)
  c = multiply_array(c, count)

  result = np.append(a, b, axis=1)
  result = np.append(result, c, axis=1)
  return result

# **Membuat Fungsi Pengujian Model dengan Citra**

**Membuat Fungsi Pembaca Dicom pada Suatu Direktori**

In [ ]:
def Read_Dicom(PathDicom):
  lstFileDicom = [] #Create an Empty List
  for dirName, subdirlist, filelist in os.walk(PathDicom):
    for filename in filelist:
      if '.dcm' in filename.lower():
        lstFileDicom.append(os.path.join(dirName, filename))
  
  # Get ref file
  RefDs = dicom.read_file(lstFileDicom[0])
  
  # Load dimensions based on the number of rows, columns, and slices (along the Z axis)
  ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFileDicom))
  
  # Load spacing values (in mm)
  ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))
  
  # The array is sized based on 'ConstPixelDims'
  ArrayDicom = np.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)
  
  # loop through all the DICOM files
  for filenameDCM in lstFileDicom:
      # read the file
      ds = dicom.read_file(filenameDCM)
      
      # store the raw image data
      ArrayDicom[:, :,lstFileDicom.index(filenameDCM)] = ds.pixel_array

  return ArrayDicom

**Melakukan Pemeriksaan GLCM pada suatu Kumpulan File Dicom**

In [ ]:
def Check_GLCM(Dicom_file):
  dissimilar = []
  energi = []
  contrast = []

  dcm_files = Dicom_file.shape[2]

  for i in range(dcm_files):
    img = Dicom_file[:, :, i]
    img = crop_center(img, 10, 10)

    glcm = greycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], 2048, symmetric=True, normed=True)

    dissimilar.append(greycoprops(glcm, 'dissimilarity')[0, 0])
    energi.append(greycoprops(glcm, 'energy')[0, 0])
    contrast.append(greycoprops(glcm, 'contrast')[0, 0])

  diss = np.asarray(dissimilar)
  en = np.asarray(energi)
  con = np.asarray(contrast)

  Dataset = [diss, energi, con]
  Dataset = np.asarray(Dataset)
  Dataset = np.transpose(Dataset)

  return Dataset

**Membuat Fungsi Prediksi pada Direktori 80 kVp**

In [ ]:
def Predict_Group(pred_material, Dicom_file, PathDicom):

  dcm_files = Dicom_file.shape[2]
  sc = StandardScaler()
  pred_material = sc.fit_transform(pred_material)
  pred_material = sc.transform(pred_material)

  y_pred_group = NN.predict(pred_material)

  lstFileDicom = [] #Create an Empty List
  for dirName, subdirlist, filelist in os.walk(PathDicom):
    for filename in filelist:
      if '.dcm' in filename.lower():
        lstFileDicom.append(os.path.join(dirName, filename))

  for j in range(dcm_files):
    y_pred_kerma_grup = np.argmax(y_pred_group[j, 0:3])
    y_pred_HVL_grup = np.argmax(y_pred_group[j, 3:6])

    A = np.asarray(lstFileDicom)

    print('-------------------------------------------------------------------------')
    print('File :', A[j])
    Prediction_Kerma(y_pred_kerma_grup)
    Prediction_HVL(y_pred_HVL_grup)
    print('-------------------------------------------------------------------------')

# **Membuka Dataset untuk diolah**
---

In [ ]:
path = '/content/drive/My Drive/Data/Dataset/New Data/Dataset_Update_4fitur.xlsx'

dataset = pd.read_excel(path, sheet_name='Main Shuffle')

In [ ]:
X = Input(dataset)
y = Labels(dataset)

# **Melakukan Pencarian Hyperparameter terbaik menggunakan GridsearchCV**

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def create_model(n_layers=1, units=10, activation='sigmoid', output_activation='sigmoid', optimizer='Adam', init_mode='glorot_uniform'):
    if isinstance(units, list):
        assert len(units) == n_layers
    else:
        units = [units] * n_layers
        
    classifier = Sequential()
 
    # Adds first hidden layer with input_dim parameter
    classifier.add(Dense(units = units[0],
                         input_dim = 6,
                         activation = activation,
                         kernel_initializer = init_mode,
                         name = 'h1'))
    
    # Adds remaining hidden layers
    for i in range(2, n_layers + 1):
        classifier.add(Dense(units = units[i-1], 
                        activation = activation, 
                        kernel_initializer = init_mode, 
                        name = 'h{}'.format(i)))
    
    # Adds output layer
    classifier.add(Dense(units = 6, 
                         activation = output_activation, 
                         kernel_initializer=init_mode, name='o'))
 
    # Compiles the model
    classifier.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', 'mean_squared_error'])
 
    return classifier

**Mencari Layer terbaik**

In [ ]:
model = KerasClassifier(build_fn=create_model, verbose=0)

layers = [6, 7, 8]
epoch = [100]
optimizer = ['Adam']
activation = ['sigmoid', 'elu']
output_activation = ['softmax']
units = [75, 100]
kernel_init = ['glorot_uniform']

param_grid = dict(n_layers=layers,
                  units=units, 
                  epochs=epoch, 
                  activation=activation,
                  output_activation=output_activation,
                  optimizer=optimizer, 
                  init_mode=kernel_init)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))